In [43]:
import numpy as np
import pandas as pd
import json
import re
import MeCab
from collections import Counter

In [138]:
def read_jasonl(filename):
    with open(filename) as f:
        return [json.loads(line.rstrip('\r\n')) for line in f.readlines()]

In [46]:
flatten = lambda l: [item for sublist in l for item in sublist if len(item) is not 0]
mecab_param = MeCab.Tagger("-Owakati -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")

In [123]:
NITE_existence_category = pd.read_csv("../data/NITE_existence_list.csv")
NITE_existence_category = NITE_existence_category.loc[NITE_existence_category.label == 1].reset_index(drop=True)
NITE_existence_category.category = NITE_existence_category.category.apply(lambda x: re.sub(r'の', '', x) if re.match(r'.+の化合物', x) else x)
NITE_existence_category.head()

,category,label
0,アルコール,1
1,アミン,1
2,フェノールエーテル,1
3,ケトン,1
4,無機化合物,1


In [4]:
with open("../data/compound_train.json") as f:
    train = json.load(f)['entry']

In [131]:
train_type_dict = dict([[str(entry['WikipediaID']), entry['Attributes']['種類']] for entry in train])
train_type_list = list(set(flatten(train_type_dict.values())))

In [ ]:
wiki_data = read_jasonl("../data/jawiki-cirrussearch-dump_of_Compound.jsonl")

In [193]:
train_wiki_doc_df = \
pd.DataFrame({
    "_id": [entry['index']['_id'] for entry in wiki_data]
    , "document": [entry['opening_text'] if ('opening_text' in entry.keys() and entry['opening_text'] != None) else entry['text'] for entry in wiki_data]
}).pipe(
    lambda x: x.loc[x._id.isin(train_type_dict.keys())]
).reset_index(drop=True)

In [12]:
train_wiki_text_df = wiki_data.loc[wiki_data._id.isin(train_type_dict.keys())]

In [195]:
#train_wiki_doc_df = train_wiki_text_df.groupby('_id').sum().rename(columns={"sentence": "document"}).reset_index()
train_wiki_doc_df.head()

,_id,document
0,2662912,ハロン (halon) は、炭化水素の水素原子（一部または全て）がハロゲン原子で置換されたハ...
1,652294,硫化窒素（りゅうかちっそ、nitrogen sulfide）は硫黄と窒素からなる無機化合物で...
2,1533538,ペンタニトロアニリン (pentanitroaniline) は、アニリンのベンゼン環上にあ...
3,184525,グリコール (glycol) はアルコールの一種（ポリオール）で、鎖式脂肪族炭化水素または環...
4,497499,N-メチル-2-ピロリドン (N-methylpyrrolidone、NMP) はラクタム構...


In [246]:
type_list = NITE_existence_category.category.tolist()

tmp_df = \
train_wiki_doc_df.apply(
    lambda x: mecab_param.parse(x.document).strip().split()
    , axis=1
).apply(
    lambda x: pd.Series(type_list, index=type_list).isin(x)
)


extraction_type_dict = \
dict(
    zip(
        train_wiki_doc_df._id.astype(str).tolist()
        , tmp_df.apply(
            lambda x: list(set(x.index.values[x.values]))
            , axis=1
        ).tolist()
    )
)

## Validation

In [247]:
result_precision = []
result_recall = []
for _id, types in extraction_type_dict.items():
    result_precision += [(_type in train_type_dict[_id]) for _type in types]
    result_recall += [(train_type in types) for train_type in train_type_dict[_id]]

In [248]:
precision = sum(result_precision) / len(result_precision)
recall = sum(result_recall) / len(result_recall)
f1 = 2 * precision * recall / (precision + recall)

print(f"Precision:{precision}")
print(f"Recall:{recall}")
print(f"f1:{f1}")

Precision:0.3027698185291309
Recall:0.4198675496688742
f1:0.3518312985571587


In [249]:
Counter(flatten(extraction_type_dict.values())).most_common()

[('有機化合物', 96),
 ('酸', 77),
 ('塩', 55),
 ('無機化合物', 53),
 ('水素', 32),
 ('アルコール', 28),
 ('アミン', 20),
 ('医薬品', 20),
 ('溶媒', 19),
 ('酸化物', 19),
 ('エーテル', 18),
 ('炭化水素', 17),
 ('タンパク質', 17),
 ('エステル', 16),
 ('塩基', 16),
 ('酢酸', 15),
 ('塩化物', 15),
 ('触媒', 15),
 ('カルボン酸', 14),
 ('芳香族化合物', 14),
 ('アルカロイド', 14),
 ('アミノ酸', 13),
 ('殺虫剤', 10),
 ('水酸化物', 10),
 ('色素', 10),
 ('炭酸塩', 10),
 ('配糖体', 9),
 ('酸化剤', 9),
 ('複素環式化合物', 9),
 ('アルカン', 8),
 ('アニオン', 8),
 ('アルケン', 8),
 ('ニトリル', 7),
 ('アミド', 7),
 ('ケトン', 7),
 ('毒', 7),
 ('過酸化物', 7),
 ('フェノール', 7),
 ('毒物', 6),
 ('臭化物', 6),
 ('染料', 6),
 ('アルデヒド', 6),
 ('爆薬', 5),
 ('脂肪酸', 5),
 ('安息香酸', 5),
 ('スルホン酸', 5),
 ('モノマー', 5),
 ('有機リン化合物', 5),
 ('尿素', 5),
 ('テルペノイド', 5),
 ('抗生物質', 4),
 ('芳香族炭化水素', 4),
 ('硫酸塩', 4),
 ('農薬', 4),
 ('糖類', 4),
 ('酸無水物', 4),
 ('チオール', 4),
 ('セスキテルペン', 4),
 ('ピリジン', 4),
 ('官能基', 4),
 ('ラクトン', 4),
 ('硝酸塩', 4),
 ('有機硫黄化合物', 4),
 ('アニリン', 3),
 ('ラクタム', 3),
 ('インドール', 3),
 ('オキソ酸', 3),
 ('脂質', 3),
 ('界面活性剤', 3),
 ('シアン化物', 3),
 ('ピラジン', 3)

In [245]:
Counter(flatten(train_type_dict.values())).most_common()

[('有機化合物', 86),
 ('無機化合物', 48),
 ('化合物', 46),
 ('酸化物', 13),
 ('誘導体', 12),
 ('塩', 9),
 ('アルカロイド', 9),
 ('フッ化物', 8),
 ('塩化物', 8),
 ('芳香族化合物', 7),
 ('アミノ酸', 7),
 ('炭酸塩', 7),
 ('カルボン酸', 7),
 ('水酸化物', 7),
 ('複素環式化合物', 6),
 ('炭化水素', 5),
 ('有機リン化合物', 5),
 ('オキソ酸', 5),
 ('臭化物', 5),
 ('有機硫黄化合物', 5),
 ('化学物質', 4),
 ('アンモニウム塩', 4),
 ('ヨウ化物', 4),
 ('アルカン', 4),
 ('色素', 3),
 ('芳香族炭化水素', 3),
 ('テルペノイド', 3),
 ('セスキテルペン', 3),
 ('中間生成物', 3),
 ('配糖体', 3),
 ('ナトリウム塩', 3),
 ('有機金属化合物', 3),
 ('アルコール', 3),
 ('酸化剤', 3),
 ('硫酸塩', 3),
 ('神経毒', 3),
 ('エーテル', 3),
 ('多環芳香族炭化水素', 2),
 ('ヒドロキシ酸', 2),
 ('蛍光色素', 2),
 ('ヌクレオシド', 2),
 ('単糖', 2),
 ('フィトケミカル', 2),
 ('植物ホルモン', 2),
 ('全長抗体', 2),
 ('ジアミン', 2),
 ('化合物半導体', 2),
 ('窒化物', 2),
 ('多糖類', 2),
 ('六フッ化物', 2),
 ('有機臭素化合物', 2),
 ('第4類危険物 第2石油類', 2),
 ('医薬品', 2),
 ('危険物第4類・第3石油類', 2),
 ('酢酸塩', 2),
 ('劇物', 2),
 ('複塩', 2),
 ('ハロゲン間化合物', 2),
 ('フェノール類', 2),
 ('有機溶媒', 2),
 ('アミン', 2),
 ('アルケン', 2),
 ('ジカルボン酸', 2),
 ('複素環式アミン', 2),
 ('ニトリル', 2),
 ('リン化物', 2),
 ('モノテルペン', 2),
